In [1]:
import geemap
import ee


In [20]:
from apps import rois, utils, satellite_params


In [5]:
roi = rois.fire_cases["Marmaris 2021"]


In [8]:
geometry = roi["region"]
date_range = roi["date_range"]


In [10]:
date_range


['2021-07-30', '2021-08-10']

In [12]:
CRS = "epsg:4326"  # Coordinate Reference System
import datetime

DAY_WINDOW = datetime.timedelta(days=6)


In [32]:
before = datetime.date.fromisoformat(date_range[0])
# before = datetime.date(before)
after = datetime.date.fromisoformat(date_range[1])
# after = datetime.date(after)


dates = {
    "prefire_start": str(before - DAY_WINDOW),
    "prefire_end": str(before),
    "postfire_start": str(after),
    "postfire_end": str(after + DAY_WINDOW),
}
dates


{'prefire_start': '2021-07-24',
 'prefire_end': '2021-07-30',
 'postfire_start': '2021-08-10',
 'postfire_end': '2021-08-16'}

In [21]:
SENTINEL = satellite_params.satellite["sentinel-2"]["name"]


In [33]:
imagery = ee.ImageCollection(SENTINEL)

prefire = imagery.filterDate(dates["prefire_start"], dates["prefire_end"]).filterBounds(
    geometry
)

postfire = imagery.filterDate(
    dates["postfire_start"], dates["postfire_end"]
).filterBounds(geometry)

pre_mos = prefire.median().clip(geometry)
post_mos = postfire.median().clip(geometry)

pre_nbr = pre_mos.normalizedDifference(["B8", "B12"])
post_nbr = post_mos.normalizedDifference(["B8", "B12"])

delta_nbr = pre_nbr.subtract(post_nbr).multiply(1000)  # why are we mutliplying by 1000?

with open("assets/sld_intervals.xml", "r", encoding="utf-8") as file:
    sld_intervals = file.read()


In [49]:
delta_nbr_sld = delta_nbr.sldStyle(sld_intervals)


In [51]:
delta_nbr_sld.getInfo()


{'type': 'Image',
 'bands': [{'id': 'vis-red',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [2, 2],
   'origin': [27, 36],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'vis-green',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [2, 2],
   'origin': [27, 36],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'vis-blue',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [2, 2],
   'origin': [27, 36],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}